<a href="https://colab.research.google.com/github/m-a-rahal/face-emotion-pfe2022/blob/main/test_2022_certitude.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !python "/content/Deep-Emotion/main.py" -

# Dataset


## Mount drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Models

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Deep_Emotion_VGG16(nn.Module):
    def __init__(self):
        '''
        Deep_Emotion class contains the network architecture.
        '''
        super(Deep_Emotion_VGG16,self).__init__()
        self.vgg16_conv1_1 = nn.Conv2d(in_channels=1, out_channels=64, kernel_size=3, padding=1)
        self.vgg16_conv1_2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1)

        self.vgg16_conv2_1 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.vgg16_conv2_2 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1)

        self.vgg16_conv3_1 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        self.vgg16_conv3_2 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)
        self.vgg16_conv3_3 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)

        self.vgg16_conv4_1 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1)
        self.vgg16_conv4_2 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)
        self.vgg16_conv4_3 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)

        self.vgg16_conv5_1 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)
        self.vgg16_conv5_2 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)
        self.vgg16_conv5_3 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)

        self.vgg16_maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.norm = nn.BatchNorm2d(10) # TODO: increase the features here or remove this layer completeley
        
        # TODO: try the old linear networks, maybe they perform better
        self.vgg16_fc1 = nn.Linear(2816, 512) #RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x512 and 25088x4096)
        self.vgg16_fc2 = nn.Linear(512, 512)
        self.vgg16_fc3 = nn.Linear(512, 7)

        self.localization = nn.Sequential(
            nn.Conv2d(1, 8, kernel_size=7),
            nn.MaxPool2d(2, stride=2),
            nn.ReLU(True),
            nn.Conv2d(8, 10, kernel_size=5),
            nn.MaxPool2d(2, stride=2),
            nn.ReLU(True)
        )

        self.fc_loc = nn.Sequential(
            nn.Linear(640, 32),
            nn.ReLU(True),
            nn.Linear(32, 3 * 2)
        )
        self.fc_loc[2].weight.data.zero_()
        self.fc_loc[2].bias.data.copy_(torch.tensor([1, 0, 0, 0, 1, 0], dtype=torch.float))

    def stn(self, x):
        xs = self.localization(x)
        xs = xs.view(-1, 640)
        theta = self.fc_loc(xs)
        theta = theta.view(-1, 2, 3)

        grid = F.affine_grid(theta, x.size())
        x = F.grid_sample(x, grid)
        return x

    def forward(self,input):
        x1 = self.stn(input)
        x2 = self.vgg16_forward(input)
        x1 = torch.flatten(x1, start_dim=1)
        x2 = torch.flatten(x2, start_dim=1)
        # concat
        x = torch.concat([x1,x2], dim=1)

        # x = torch.flatten(x, start_dim=1)
        x = F.relu(self.vgg16_fc1(x))
        x = F.dropout(x, 0.5) #dropout was included to combat overfitting
        x = F.relu(self.vgg16_fc2(x))
        x = F.dropout(x, 0.5)
        x = self.vgg16_fc3(x)
        # NOTE: DO NOT ADD SOFTMAX AT THE END, here's why :  https://stackoverflow.com/questions/67466531/cnn-pytorch-only-batches-of-spatial-targets-supported-error
        
        return x

    def vgg16_forward(self, x):
        x = F.relu(self.vgg16_conv1_1(x))
        x = F.relu(self.vgg16_conv1_2(x))
        x = self.vgg16_maxpool(x)
        x = F.relu(self.vgg16_conv2_1(x))
        x = F.relu(self.vgg16_conv2_2(x))
        x = self.vgg16_maxpool(x)
        x = F.relu(self.vgg16_conv3_1(x))
        x = F.relu(self.vgg16_conv3_2(x))
        x = F.relu(self.vgg16_conv3_3(x))
        x = self.vgg16_maxpool(x)
        x = F.relu(self.vgg16_conv4_1(x))
        x = F.relu(self.vgg16_conv4_2(x))
        x = F.relu(self.vgg16_conv4_3(x))
        x = self.vgg16_maxpool(x)
        x = F.relu(self.vgg16_conv5_1(x))
        x = F.relu(self.vgg16_conv5_2(x))
        x = F.relu(self.vgg16_conv5_3(x))
        x = self.vgg16_maxpool(x)
        return x

print("Model archticture: ", Deep_Emotion_VGG16())

In [ ]:
class Simple_VGG16(nn.Module):
    def __init__(self):
        '''
        Deep_Emotion class contains the network architecture.
        '''
        super(Simple_VGG16,self).__init__()
        self.vgg16_conv1_1 = nn.Conv2d(in_channels=1, out_channels=64, kernel_size=3, padding=1)
        self.vgg16_conv1_2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1)

        self.vgg16_conv2_1 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.vgg16_conv2_2 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1)

        self.vgg16_conv3_1 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        self.vgg16_conv3_2 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)
        self.vgg16_conv3_3 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)

        self.vgg16_conv4_1 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1)
        self.vgg16_conv4_2 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)
        self.vgg16_conv4_3 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)

        self.vgg16_conv5_1 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)
        self.vgg16_conv5_2 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)
        self.vgg16_conv5_3 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)

        self.vgg16_maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.norm = nn.BatchNorm2d(10) # TODO: increase the features here or remove this layer completeley
        
        # TODO: try the old linear networks, maybe they perform better
        self.vgg16_fc1 = nn.Linear(512, 512)
        self.vgg16_fc2 = nn.Linear(512, 128)
        self.vgg16_fc3 = nn.Linear(128, 7)

        self.localization = nn.Sequential(
            nn.Conv2d(1, 8, kernel_size=7),
            nn.MaxPool2d(2, stride=2),
            nn.ReLU(True),
            nn.Conv2d(8, 10, kernel_size=5),
            nn.MaxPool2d(2, stride=2),
            nn.ReLU(True)
        )

        self.fc_loc = nn.Sequential(
            nn.Linear(640, 32),
            nn.ReLU(True),
            nn.Linear(32, 3 * 2)
        )
        self.fc_loc[2].weight.data.zero_()
        self.fc_loc[2].bias.data.copy_(torch.tensor([1, 0, 0, 0, 1, 0], dtype=torch.float))

    def stn(self, x):
        xs = self.localization(x)
        xs = xs.view(-1, 640)
        theta = self.fc_loc(xs)
        theta = theta.view(-1, 2, 3)

        grid = F.affine_grid(theta, x.size())
        x = F.grid_sample(x, grid)
        return x

    def forward(self,input):
        #x1 = self.stn(input)
        x = self.vgg16_forward(input)
        x = torch.flatten(x, start_dim=1)
        #x = torch.flatten(x, start_dim=1)
        x = F.relu(self.vgg16_fc1(x))
        x = F.dropout(x, 0.5) #dropout was included to combat overfitting
        x = F.relu(self.vgg16_fc2(x))
        x = F.dropout(x, 0.5)
        x = self.vgg16_fc3(x)
        # NOTE: DO NOT ADD SOFTMAX AT THE END, here's why :  https://stackoverflow.com/questions/67466531/cnn-pytorch-only-batches-of-spatial-targets-supported-error
        
        return x

    def vgg16_forward(self, x):
        x = F.relu(self.vgg16_conv1_1(x))
        x = F.relu(self.vgg16_conv1_2(x))
        x = self.vgg16_maxpool(x)
        x = F.relu(self.vgg16_conv2_1(x))
        x = F.relu(self.vgg16_conv2_2(x))
        x = self.vgg16_maxpool(x)
        x = F.relu(self.vgg16_conv3_1(x))
        x = F.relu(self.vgg16_conv3_2(x))
        x = F.relu(self.vgg16_conv3_3(x))
        x = self.vgg16_maxpool(x)
        x = F.relu(self.vgg16_conv4_1(x))
        x = F.relu(self.vgg16_conv4_2(x))
        x = F.relu(self.vgg16_conv4_3(x))
        x = self.vgg16_maxpool(x)
        x = F.relu(self.vgg16_conv5_1(x))
        x = F.relu(self.vgg16_conv5_2(x))
        x = F.relu(self.vgg16_conv5_3(x))
        x = self.vgg16_maxpool(x)
        return x

print("Model archticture: ", Deep_Emotion_VGG16())

In [ ]:
class Deep_Emotion(nn.Module):
    def __init__(self):
        '''
        Deep_Emotion class contains the network architecture.
        '''
        super(Deep_Emotion,self).__init__()
        self.conv1 = nn.Conv2d(1,10,3)
        self.conv2 = nn.Conv2d(10,10,3)
        self.pool2 = nn.MaxPool2d(2,2)

        self.conv3 = nn.Conv2d(10,10,3)
        self.conv4 = nn.Conv2d(10,10,3)
        self.pool4 = nn.MaxPool2d(2,2)

        self.norm = nn.BatchNorm2d(10)

        self.fc1 = nn.Linear(810,50)
        self.fc2 = nn.Linear(50,7)

        self.localization = nn.Sequential(
            nn.Conv2d(1, 8, kernel_size=7),
            nn.MaxPool2d(2, stride=2),
            nn.ReLU(True),
            nn.Conv2d(8, 10, kernel_size=5),
            nn.MaxPool2d(2, stride=2),
            nn.ReLU(True)
        )

        self.fc_loc = nn.Sequential(
            nn.Linear(640, 32),
            nn.ReLU(True),
            nn.Linear(32, 3 * 2)
        )
        self.fc_loc[2].weight.data.zero_()
        self.fc_loc[2].bias.data.copy_(torch.tensor([1, 0, 0, 0, 1, 0], dtype=torch.float))

    def stn(self, x):
        xs = self.localization(x)
        xs = xs.view(-1, 640)
        theta = self.fc_loc(xs)
        theta = theta.view(-1, 2, 3)

        grid = F.affine_grid(theta, x.size())
        x = F.grid_sample(x, grid)
        return x

    def forward(self,input):
        out = self.stn(input)

        out = F.relu(self.conv1(out))
        out = self.conv2(out)
        out = F.relu(self.pool2(out))

        out = F.relu(self.conv3(out))
        out = self.norm(self.conv4(out))
        out = F.relu(self.pool4(out))

        out = F.dropout(out)
        out = out.view(-1, 810)
        out = F.relu(self.fc1(out))
        out = self.fc2(out)

        return out

In [ ]:
class Shallow_Emotion(nn.Module):
    def __init__(self):
        '''
        Deep_Emotion class contains the network architecture.
        '''
        super(Shallow_Emotion,self).__init__()
        self.conv1 = nn.Conv2d(1,10,3)
        self.conv2 = nn.Conv2d(10,10,3)
        self.pool2 = nn.MaxPool2d(2,2)

        self.conv3 = nn.Conv2d(10,10,3)
        self.conv4 = nn.Conv2d(10,10,3)
        self.pool4 = nn.MaxPool2d(2,2)

        # self.norm = nn.BatchNorm2d(10)

        self.fc1 = nn.Linear(810,50)
        self.fc2 = nn.Linear(50,7)

    def forward(self,input):
        out = F.relu(self.conv1(input))
        out = self.conv2(out)
        out = F.relu(self.pool2(out))

        out = F.relu(self.conv3(out))
        out = self.conv4(out)
        out = F.relu(self.pool4(out))

        out = F.dropout(out)
        out = out.view(-1, 810)
        out = F.relu(self.fc1(out))
        out = self.fc2(out)

        return out

In [ ]:
class Deep_Emotion_with_concat(nn.Module):
    def __init__(self):
        '''
        Deep_Emotion class contains the network architecture.
        '''
        super(Deep_Emotion_with_concat,self).__init__()
        self.conv1 = nn.Conv2d(1,10,3)
        self.conv2 = nn.Conv2d(10,10,3)
        self.pool2 = nn.MaxPool2d(2,2)

        self.conv3 = nn.Conv2d(10,10,3)
        self.conv4 = nn.Conv2d(10,10,3)
        self.pool4 = nn.MaxPool2d(2,2)

        self.norm = nn.BatchNorm2d(10)

        self.fc1 = nn.Linear(3114,1024) #mat1 and mat2 shapes cannot be multiplied (128x3114 and 810x50)
        self.fc2 = nn.Linear(1024,7)

        self.localization = nn.Sequential(
            nn.Conv2d(1, 8, kernel_size=7),
            nn.MaxPool2d(2, stride=2),
            nn.ReLU(True),
            nn.Conv2d(8, 10, kernel_size=5),
            nn.MaxPool2d(2, stride=2),
            nn.ReLU(True)
        )

        self.fc_loc = nn.Sequential(
            nn.Linear(640, 32),
            nn.ReLU(True),
            nn.Linear(32, 3 * 2)
        )
        self.fc_loc[2].weight.data.zero_()
        self.fc_loc[2].bias.data.copy_(torch.tensor([1, 0, 0, 0, 1, 0], dtype=torch.float))

    def stn(self, x):
        xs = self.localization(x)
        xs = xs.view(-1, 640)
        theta = self.fc_loc(xs)
        theta = theta.view(-1, 2, 3)

        grid = F.affine_grid(theta, x.size())
        x = F.grid_sample(x, grid)
        return x

    def forward(self,input):
        x1 = self.stn(input)

        x2 = F.relu(self.conv1(input))
        x2 = self.conv2(x2)
        x2 = F.relu(self.pool2(x2))

        x2 = F.relu(self.conv3(x2))
        x2 = self.norm(self.conv4(x2))
        x2 = F.relu(self.pool4(x2))

        x2 = F.dropout(x2)
        x2 = x2.view(-1, 810)
        
        x1 = torch.flatten(x1, start_dim=1)
        out = torch.concat([x1,x2], dim=1)
        
        out = F.relu(self.fc1(out))
        out = self.fc2(out)

        return out

In [ ]:
class Deep_Emotion_with_mult(nn.Module):
    def __init__(self):
        '''
        Deep_Emotion class contains the network architecture.
        '''
        super(Deep_Emotion_with_mult,self).__init__()
        self.conv1 = nn.Conv2d(1,10,3)
        self.conv2 = nn.Conv2d(10,10,3)
        self.pool2 = nn.MaxPool2d(2,2)

        self.conv3 = nn.Conv2d(10,10,3)
        self.conv4 = nn.Conv2d(10,10,3)
        self.pool4 = nn.MaxPool2d(2,2)

        self.norm = nn.BatchNorm2d(10)

        self.fc1 = nn.Linear(23040,1024) #mat1 and mat2 shapes cannot be multiplied (128x23040 and 3114x1024)
        self.fc2 = nn.Linear(1024,7)

        self.localization = nn.Sequential(
            nn.Conv2d(1, 8, kernel_size=7),
            nn.MaxPool2d(2, stride=2),
            nn.ReLU(True),
            nn.Conv2d(8, 10, kernel_size=5),
            nn.MaxPool2d(2, stride=2),
            nn.ReLU(True)
        )

        self.fc_loc = nn.Sequential(
            nn.Linear(640, 32),
            nn.ReLU(True),
            nn.Linear(32, 3 * 2)
        )
        self.fc_loc[2].weight.data.zero_()
        self.fc_loc[2].bias.data.copy_(torch.tensor([1, 0, 0, 0, 1, 0], dtype=torch.float))

    def stn(self, x):
        xs = self.localization(x)
        xs = xs.view(-1, 640)
        theta = self.fc_loc(xs)
        theta = theta.view(-1, 2, 3)

        grid = F.affine_grid(theta, x.size())
        x = F.grid_sample(x, grid)
        return x

    def forward(self,input):
        x1 = self.stn(input)

        x2 = F.relu(self.conv1(input))
        x2 = self.conv2(x2)
        x2 = F.relu(self.pool2(x2))

        x2 = F.relu(self.conv3(x2))
        x2 = self.norm(self.conv4(x2))
        x2 = F.relu(self.pool4(x2))

        x2 = F.dropout(x2)
        x2 = F.interpolate(x2, size=(48,48), mode='linear')
        out = F.att(x1,x2)
        out = torch.flatten(out, start_dim=1)
        
        #x2 = x2.view(-1, 810)
        
        #x1 = torch.flatten(x1, start_dim=1)
        
        
        out = F.relu(self.fc1(out))
        out = self.fc2(out)

        return out

In [ ]:
class Umer2021(nn.Module):
    def __init__(self):
        super(Umer2021,self).__init__()
        self.nn = nn.Sequential(
            # Bllock 1
            nn.Conv2d(1,64,3), # nn.Conv2d(in_channels=1, out_channels=64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.MaxPool2d(2,2),
            nn.Dropout(),
            # Block 2
            nn.Conv2d(64,128,5),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.MaxPool2d(2,2),
            nn.Dropout(),
            # Block 3
            nn.Conv2d(128,128,5),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.MaxPool2d(2,2),
            nn.Dropout(),
            # Block 4
            # nn.Conv2d(128,512,3),
            # nn.BatchNorm2d(128),
            # nn.ReLU(True),
            # nn.MaxPool2d(2,2),
            # nn.Dropout(),
            # # Block 5
            # nn.Conv2d(512, 512, 3),
            # nn.BatchNorm2d(128),
            # nn.ReLU(True),
            # nn.MaxPool2d(2, 2),
            # nn.Dropout(),
            # # Block 6
            # nn.Conv2d(512, 1024, 3),
            # nn.BatchNorm2d(128),
            # nn.ReLU(True),
            # nn.MaxPool2d(2, 2),
            # nn.Dropout(),


            # Flat layers
            nn.Flatten(),
            nn.Linear(512,256),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(256, 512),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(512,7)
        )
    
    def forward(self,input):
        x = self.nn(input)
        return x

# reading data and definitions


In [ ]:
import torch
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, X, Y, trans = None):
        assert len(X) == len(Y), "X and Y must have same size, make sure you're using the right pair"
        self.X = X
        self.Y = Y
        self.transform = trans

    def __len__(self):
        return len(self.X)
    def __getitem__(self, index):
        image = self.X[index]
        label = self.Y[index] #'Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral'#
        if self.transform is not None:
            image = self.transform(image)
        return image, label

In [ ]:
from __future__ import print_function
import argparse
import numpy  as np
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import transforms

#from data_loaders import Plain_Dataset, eval_data_dataloader
from torch.utils.tensorboard import SummaryWriter
from PIL import Image

data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/dataset/fer2013.csv")
data.head()

groups = [g for _, g in data.groupby('Usage')]
# print([x for x,_ in data.groupby('Usage')])
train = groups[2]
val = groups[1]
test = groups[0]
train = train.drop(labels=['Usage'], axis=1)
val = val.drop(labels=['Usage'], axis=1)
test = test.drop(labels=['Usage'], axis=1)

Y_train = train["emotion"]
Y_val = val["emotion"]
Y_test = test["emotion"]


X_train = train["pixels"]
X_val = val["pixels"]
X_test = test["pixels"]

def preprocess(X):
    X = np.array([np.fromstring(image, np.uint8, sep=' ') for image in X]).astype(np.float32)
    X = X/255.0
    X = X.reshape(-1, 48, 48, 1)
    return X

X_train = preprocess(X_train)
X_val = preprocess(X_val)
X_test = preprocess(X_test)
Y_train = np.array(Y_train)
Y_val = np.array(Y_val)
Y_test = np.array(Y_test)



### transforms

In [ ]:
import torchvision.transforms as T
# Data transforms (Gray Scaling & data augmentation)
train_transform = T.Compose([T.ToTensor(), T.Normalize((0.5,),(0.5,)), T.RandomHorizontalFlip(),
                             T.RandomAdjustSharpness(0.9, p=0), T.RandomRotation(5)])#, tt.RandomRotation(10)])
transform =  T.Compose([T.ToTensor(),T.Normalize((0.5,),(0.5,))])

# ------------------------------------------------------------------
train_dataset = CustomDataset(X_train, Y_train, train_transform)
test_dataset = CustomDataset(X_test, Y_test, transform)
val_dataset = CustomDataset(X_val, Y_val, transform)

### train function



In [ ]:
def Train(net,epochs,train_loader,val_loader,criterion,optmizer,device, writer):
    '''
    Training Loop
    '''
    print("===================================Start Training===================================")
    for e in range(epochs):
        train_loss = 0
        validation_loss = 0
        train_correct = 0
        val_correct = 0
        # Train the model  #
        net.train()
        for i, (data, labels) in enumerate(train_loader):
            data, labels = data.to(device), labels.to(device)
            optmizer.zero_grad()
            outputs = net(data)
            loss = criterion(outputs,labels)
            loss.backward()
            optmizer.step()
            train_loss += loss.item()
            _, preds = torch.max(outputs,1)
            train_correct += torch.sum(preds == labels.data)
            print('\r', 'Epoch: %d | score : %.2f' \
                %(e+1, train_correct/(i+0.0001)), end = '') 
        print('\r', end='')
        #validate the model#
        net.eval()
        for data,labels in val_loader:
            data, labels = data.to(device), labels.to(device)
            val_outputs = net(data)
            val_loss = criterion(val_outputs, labels)
            validation_loss += val_loss.item()
            _, val_preds = torch.max(val_outputs,1)
            val_correct += torch.sum(val_preds == labels.data)
            

        train_loss = train_loss/len(train_dataset)
        train_acc = train_correct.double() / len(train_dataset)
        validation_loss =  validation_loss / len(val_dataset)
        val_acc = val_correct.double() / len(val_dataset)
        print('Epoch: {} \tTraining Loss: {:.8f} \tValidation Loss {:.8f} \tTraining Acuuarcy {:.3f}% \tValidation Acuuarcy {:.3f}%'
                                                           .format(e+1, train_loss,validation_loss,train_acc * 100, val_acc*100))
        writer.add_scalar("Loss/train", train_loss, e)
        writer.add_scalar("Accuracy/train", train_acc, e)
        writer.add_scalar("Loss/val", validation_loss, e)
        writer.add_scalar("Accuracy/val", val_acc, e)

    torch.save(net.state_dict(),'deep_emotion-{}-{}-{}.pt'.format(epochs,batch_size,lr))
    print("===================================Training Finished===================================")

# Training

In [ ]:
# hyperparams
epochs = 100
lr = 0.005
use_class_weights = False
batch_size = 128
# network
net = Deep_Emotion_with_mult()

# PyTorch data loaders
train_loader = DataLoader(train_dataset, batch_size, num_workers=2, pin_memory=True, shuffle=True)
val_loader   = DataLoader(val_dataset,   batch_size, num_workers=2, pin_memory=True)
test_loader  = DataLoader(test_dataset,  batch_size, num_workers=2, pin_memory=True)



import gc
gc.collect()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net.to(device)
traincsv_file = '/content/datasets/train.csv'
validationcsv_file = '/content/datasets/val.csv'
train_img_dir = '/content/datasets/train/'
validation_img_dir = '/content/datasets/val/'

transformation= transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,),(0.5,))])
#train_dataset= Plain_Dataset(csv_file=traincsv_file, img_dir = train_img_dir, datatype = 'train', transform = transformation)
#validation_dataset= Plain_Dataset(csv_file=validationcsv_file, img_dir = validation_img_dir, datatype = 'val', transform = transformation)
#train_loader= DataLoader(train_dataset,batch_size=batchsize,shuffle = True,num_workers=0)
#val_loader=   DataLoader(validation_dataset,batch_size=batchsize,shuffle = True,num_workers=0)
writer = SummaryWriter('runs/fer2013_experiment_1')
cweights = [1.02660468, 9.40661861, 1.00104606, 0.56843877, 0.84912748, 1.29337298, 0.82603942]
class_weights = torch.FloatTensor(cweights).cuda()
if use_class_weights:
    criterion= nn.CrossEntropyLoss(weight=class_weights)
else:
    criterion= nn.CrossEntropyLoss()
optmizer= optim.Adam(net.parameters(),lr= lr)
Train(net,epochs, train_loader, val_loader, criterion, optmizer, device, writer)


===================================Start Training===================================


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:4066: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  "Default grid_sample and affine_grid behavior has changed "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:4004: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  "Default grid_sample and affine_grid behavior has changed "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3635: UserWarning: Default upsampling behavior when mode=linear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See 

NotImplementedError: ignored

In [ ]:
torch.save(net, "/content/drive/MyDrive/Colab Notebooks/Face emotion detection/Models/tmp13548")

In [ ]:
"""latest execution : concat model
"Epoch: 88 	Training Loss: 0.00890384 	Validation Loss 0.01008887 	Training Acuuarcy 56.484% 	Validation Acuuarcy 52.327%
Epoch: 89 	Training Loss: 0.00892394 	Validation Loss 0.00995875 	Training Acuuarcy 56.703% 	Validation Acuuarcy 53.246%
Epoch: 90 	Training Loss: 0.00888090 	Validation Loss 0.01016509 	Training Acuuarcy 56.906% 	Validation Acuuarcy 53.608%
Epoch: 91 	Training Loss: 0.00891105 	Validation Loss 0.01013295 	Training Acuuarcy 57.090% 	Validation Acuuarcy 52.522%
Epoch: 92 	Training Loss: 0.00888799 	Validation Loss 0.01044168 	Training Acuuarcy 56.832% 	Validation Acuuarcy 52.271%
Epoch: 93 	Training Loss: 0.00889699 	Validation Loss 0.01008596 	Training Acuuarcy 56.919% 	Validation Acuuarcy 53.190%
Epoch: 94 	Training Loss: 0.00888363 	Validation Loss 0.01012353 	Training Acuuarcy 56.860% 	Validation Acuuarcy 53.302%
Epoch: 95 	Training Loss: 0.00882737 	Validation Loss 0.01007196 	Training Acuuarcy 57.027% 	Validation Acuuarcy 53.469%
Epoch: 96 	Training Loss: 0.00886491 	Validation Loss 0.00996131 	Training Acuuarcy 56.735% 	Validation Acuuarcy 52.856%
Epoch: 97 	Training Loss: 0.00886380 	Validation Loss 0.00995262 	Training Acuuarcy 57.041% 	Validation Acuuarcy 53.775%
Epoch: 98 	Training Loss: 0.00886352 	Validation Loss 0.00999040 	Training Acuuarcy 57.362% 	Validation Acuuarcy 54.166%
Epoch: 99 	Training Loss: 0.00880713 	Validation Loss 0.01013636 	Training Acuuarcy 57.135% 	Validation Acuuarcy 52.967%
Epoch: 100 	Training Loss: 0.00889800 	Validation Loss 0.01013484 	Training Acuuarcy 56.951% 	Validation Acuuarcy 52.995%
===================================Training Finished==================================="""